## 0. 라이브러리

In [52]:
import os
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import json
from soynlp.normalizer import *
from tqdm import tqdm
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## 1. 데이터 불러오기

In [53]:
train_data_path ="~/aiffel/dktc/data2/train2.csv"
train_data = pd.read_csv(train_data_path,index_col=0)
train_data

,class,conversation
idx,,
1,일반 대화,오늘 날씨 어때?\n맑고 따뜻해.\n좋네! 주말에 계획 있어?\n등산 갈 생각이야....
2,일반 대화,오늘 날씨 어때?\n맑고 따뜻해.\n좋네! 주말에 계획 있어?\n등산 갈 생각이야....
3,기타 괴롭힘 대화,너 피아노 좀 쳐봐.\n싫어.\n왜 손가락 없다고 유세 떠는 거야?\n이씨.\n비행...
4,갈취 대화,요즘에는 액수가 작네?\n미안해 요즘에 용돈이 작아\n그게 나랑 무슨 상관이야?\n...
5,기타 괴롭힘 대화,야 이거봐 완전 길동이 닯음\n 진짜네 \n야 그러지마.\n왜 똑같구만 원숭이 \n...
...,...,...
4946,직장 내 괴롭힘 대화,어이 신병.\n이병 김범례.\n와봐.\n네.\n네? 뒤질래?\n.자.잘못들었습니다?...
4947,직장 내 괴롭힘 대화,오늘 회의 안건인 길동프로그램의 출연자는 누구로 할 것인가에 대해 모두 의견 내주시...
4948,직장 내 괴롭힘 대화,야 열심히들 해라 새끼들아\n넵 감사해요 부장님\n야 내가 언제 너 한테 말했냐\n...


## 2. 데이터 준비 (Data preparation)
### 2.1-1 전처리 함수 정의

In [54]:
def preprocess_sentence(sentence):
    # synolp
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    # base preprocess
    sentence = re.sub(r'([^a-zA-Zㄱ-ㅎ가-힣?.!,])', " ", sentence)
    sentence = re.sub(r'!+', '!', sentence)
    sentence = re.sub(r'\?+', '?', sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # 엔터 구분 (\n)
    sentence = sentence.replace("\n", " ")
    sentence = sentence.strip()
    return sentence

### 2.1-2 전처리 함수 적용

In [55]:
# 학습할 문장이 담길 배열
sentences = []

for val in tqdm(train_data['conversation']):
    sentences.append(preprocess_sentence(val))

100%|██████████| 4950/4950 [00:01<00:00, 3290.96it/s]


### 2.2 최대 길이 지정

In [56]:
MAX_LEN = 200

### 2.3 class(label) 인코딩

In [57]:
from sklearn.preprocessing import LabelEncoder

CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])
labels = train_data['class']

len(labels)

4950

In [58]:
class_mapping = {class_name: encoder.transform([class_name])[0] for class_name in CLASS_NAMES}
print("Class mapping:", class_mapping)

Class mapping: {'협박 대화': 4, '갈취 대화': 0, '직장 내 괴롭힘 대화': 3, '기타 괴롭힘 대화': 1, '일반 대화': 2}


### 2.4 train-val

In [59]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42)



In [60]:
### train data 증강

# 데이터 증강 함수

def random_deletion(words, p=0.3):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words) - 1)
        return [words[rand_int]]

    return "".join(new_words)

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = (
        new_words[random_idx_2],
        new_words[random_idx_1],
    )
    return new_words


def random_swap(words, n=3):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words


print("before data augmentation: ", len(train_sentences))

train_splted = pd.DataFrame({ "sentence": train_sentences, "class": train_labels })

# random deletion
train_splted_rd = train_splted.copy()
train_splted_rd["sentence"] = train_splted_rd["sentence"].apply(random_deletion)

# random swap
train_splted_rs = train_splted.copy()
#train_splted_rs["sentence"] = random_swap(train_splted_rs["sentence"])

# with data augmentation
train_concated = pd.concat([train_splted , train_splted_rd , train_splted_rs])

print("after data augmentation: ", len(train_concated))

train_concated

before data augmentation:  3960
after data augmentation:  11880


,sentence,class
idx,,
3667,오늘 날씨 어때 ? 맑고 따뜻해 . 좋네 ! 주말에 계획 있어 ? 등산 갈 생각이야...,2
1021,사장님 . 배달이 분까지인데 분이나 늦었잖아요 . 죄송합니다 . 배달원한테 전달을 ...,1
3227,이번 달 월급 들어오면 갚는다며 . 들어오자마자 다 뜯겨서 그래 . 미안해 . 그 ...,0
1030,야 우리 내일 현장학습인 거 알지 ? 응 ? 알지 . 그럼 니가 내 대신 내도시락도...,1
2909,여기가 어딘가요 ? 야 있는 돈 다 내놔 . 맞기 싫으면 아니 댁은 누구신대 다짜고...,0
...,...,...
4427,야 일 제대로 안해 ? 아니 물량수준이 희망사항 수준인데 그래서 뭐 임마 그 많은 ...,3
467,역시 난 니가 올 줄 알았어 . 우리가족을 인질로 잡아 ? 니가 그러고도 사람이야 ...,4
3093,밤마다 뭘하는 지 시끄러워서 잠을 못자겠다고 ! 밤에 뭐 하긴 자지 . 자는 데 왜...,4


## 3. 모델
### 3.1-1 토크나이저 정의

In [61]:
# BERT 토크나이저와 모델 준비
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

### 3.1-2 토크나이저 적용

In [62]:
# 데이터셋을 BERT 입력 형식으로 변환
train_encodings = tokenizer(train_sentences, truncation=True, padding=True, max_length=MAX_LEN) # 뒤쪽에 패딩
val_encodings = tokenizer(val_sentences, truncation=True, padding=True, max_length=MAX_LEN)

### 3.2 모델 준비

In [63]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 3.3 파라미터

In [71]:
BATCH_SIZE = 8
lr = 5e-5
EPOCH = 10

### 3.4 TF 데이터셋 생성

In [72]:
# TensorFlow 데이터셋 생성
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(100).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(BATCH_SIZE)


### 3.5 모델 컴파일

In [73]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'],
             run_eagerly=False)

### 3.6 모델 훈련

### 3.6-1 콜백 설정

In [74]:
early_stopping = EarlyStopping(
    monitor='val_loss',    # 검증 손실을 모니터링
    patience=3,            # 3 에포크 동안 개선되지 않으면 중지
    restore_best_weights=True  # 최상의 가중치를 복원
)

checkpoint = ModelCheckpoint(
    filepath='best_model_2.weights.h5',  # 모델 가중치를 저장할 파일 경로
    monitor='val_loss',        # 검증 손실을 모니터링
    save_best_only=True,       # 최상의 모델만 저장
    save_weights_only=True,   # 저장 (가중치)
    mode='min',                # 'val_loss'가 최소일 때 저장
    verbose=1                  # 저장 시 로그 출력
)

### 3.6-2 모델 훈련

In [75]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=EPOCH,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/10
495/495 [==============================] - 226s 429ms/step - loss: 1.5570 - accuracy: 0.2424 - val_loss: 1.6075 - val_accuracy: 0.2253

Epoch 00001: val_loss improved from inf to 1.60748, saving model to best_model_2.weights.h5
Epoch 2/10
495/495 [==============================] - 210s 423ms/step - loss: 1.6163 - accuracy: 0.2015 - val_loss: 1.6087 - val_accuracy: 0.2253

Epoch 00002: val_loss did not improve from 1.60748
Epoch 3/10
495/495 [==============================] - 209s 423ms/step - loss: 1.6136 - accuracy: 0.2144 - val_loss: 1.6114 - val_accuracy: 0.2253

Epoch 00003: val_loss did not improve from 1.60748
Epoch 4/10
495/495 [==============================] - 209s 422ms/step - loss: 1.6133 - accuracy: 0.2035 - val_loss: 1.6092 - val_accuracy: 0.2253

Epoch 00004: val_loss did not improve from 1.60748


### 3.7 모델 평가

In [76]:
# 모델 평가
evaluation = model.evaluate(val_dataset)
print("평가 결과:", evaluation)

124/124 [==============================] - 15s 121ms/step - loss: 1.6075 - accuracy: 0.2253
평가 결과: [1.6074848175048828, 0.22525252401828766]


## 4. 모델 적용

In [77]:
test_data_path = "~/aiffel/dktc/data2/test.json"

with open(test_data_path, "r", encoding="utf-8") as json_file:
    test = json.load(json_file)

FileNotFoundError: [Errno 2] No such file or directory: '~/aiffel/dktc/data2/test.json'

In [35]:
import numpy as np

test_predicst = list()

for key in test:
    test_sentence = test['text']
    
    test_encodings = tokenizer(test_sentence, truncation=True, padding=True, max_length=128, return_tensors="tf")
    
    test_predictions = model.predict({
        "input_ids": test_encodings["input_ids"],
        "token_type_ids": test_encodings["token_type_ids"],
        "attention_mask": test_encodings["attention_mask"]
    }) # [ 0.7805823,  2.6188664, -2.0281641, -0.9672525]
    test_class_probabilities = tf.nn.softmax(test_predictions.logits, axis=-1).numpy() # [[0.13297564 0.8358507  0.00801584 0.02315779]]
    test_predicted_class = np.argmax(test_class_probabilities, axis=1) # [ 1 ]
    test_predicst.append(test_predicted_class[0])

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
def labelnum_to_text(x):
    if x == 1 : # 갈취
        return '01'
    if x == 2 : # 직장
        return '02'
    if x == 3 : # 기타
        return '03'
    if x == 0 : # 협박 
        return '00'
    
submission = pd.DataFrame({'class':test_predicst}, index=list(test.keys()))

submission['class'] = submission['class'].apply(labelnum_to_text)
submission

In [ ]:
#  submission.to_csv('~/aiffel/dktc/sub/base_sub.csv')